In [18]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential,Model,initializers,layers,Input
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [67]:
# full-pre-activation 방식
def Convblock(x,k):
    x1=layers.BatchNormalization()(x)
    x1=layers.ReLU()(x1)
    x1=layers.Conv2D(filters=4*k,kernel_size=(1,1),strides=(1,1),padding='same')(x1)
    
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)
    x1=layers.Conv2D(filters=k,kernel_size=(3,3),strides=(1,1),padding='same')(x1)
    
    x2=tf.keras.layers.Concatenate()([x,x1])

    return x2

def DenseBlock_1(x,k):
    for i in range(6):
        x_temp=Convblock(x,k)
        x=x_temp
    return x

def DenseBlock_2(x,k):
    for i in range(12):
        x_temp=Convblock(x,k)
        x=x_temp
    return x


def DenseBlock_3(x,k):
    for i in range(48):
        x_temp=Convblock(x,k)
        x=x_temp
    return x

def DenseBlock_4(x,k):
    for i in range(32):
        x_temp=Convblock(x,k)
        x=x_temp
    return x

def Transition_layer(x,k,theta):
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    x=layers.Conv2D(filters=(256+3*k)*theta,kernel_size=(1,1),use_bias=False,strides=(1,1),padding='same',)(x)
    x=layers.AveragePooling2D(pool_size=(2,2),strides=(2,2))(x)
    return x

def Classification_layer(x):
    x=layers.GlobalAveragePooling2D()(x)
    x=layers.Dense(units=10,activation='softmax')(x)
    return x

In [68]:
input=Input(shape=(224,224,3))
x=layers.BatchNormalization()(input)
x=layers.ReLU()(x)
x=layers.Conv2D(filters=256,kernel_size=(7,7),strides=(2,2),padding='same')(x)
x=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)

x=DenseBlock_1(x,32)
x=Transition_layer(x,32,0.2)

x=DenseBlock_2(x,32)
x=Transition_layer(x,32,0.2)

x=DenseBlock_3(x,32)
x=Transition_layer(x,32,0.2)

x=DenseBlock_4(x,32)

output=Classification_layer(x)

model = tf.keras.models.Model(input,output)

In [58]:
len(model.layers)

705

In [62]:
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 batch_normalization_2410 (Batc  (None, 224, 224, 3)  12         ['input_21[0][0]']               
 hNormalization)                                                                                  
                                                                                                  
 re_lu_2410 (ReLU)              (None, 224, 224, 3)  0           ['batch_normalization_2410[0][0]'
                                                                 ]                         

In [ ]:
plot_model(model,to_file='DenseNet.png',show_shapes=True)

In [64]:
import tensorflow as tf
import numpy as np

(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar10.load_data()

# Normalization
x_train=x_train/255.

x_test=x_test/255.

x_train.shape

x_train = tf.image.resize(x_train, [224,224]) 

x_test = tf.image.resize(x_test, [224,224]) 

y_train = tf.keras.utils.to_categorical(np.array(y_train))
y_test = tf.keras.utils.to_categorical(np.array(y_test))

In [69]:
model.compile(optimizer='adam',loss='categorical_crossentropy' ,metrics=['acc'])

In [70]:
with tf.device("/device:CPU:0"):
    history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=1,batch_size=8,verbose=1)

  20/6250 [..............................] - ETA: 7:24:30 - loss: 5.3376 - acc: 0.0750